{/* cspell:ignore hyperparameters */}

# Singularity Machine Learning - Classification: A Qiskit Function by Multiverse Computing

<LegacyContent>
<Admonition type="note">
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/multiverse-computing-singularity)
</Admonition>
</LegacyContent>
<CloudContent>
<Admonition type="note">
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/multiverse-computing-singularity)
</Admonition>
</CloudContent>

## Overview

With the "Singularity Machine Learning - Classification" function, you can solve real-world machine learning problems on quantum hardware without requiring quantum expertise. This Application function, based on ensemble methods, is a hybrid classifier. It leverages classical methods like boosting, bagging, and stacking for initial ensemble training. Subsequently, quantum algorithms such as variational quantum eigensolver (VQE) and quantum approximate optimization algorithm (QAOA) are employed to enhance the trained ensemble's diversity, generalization capabilities, and overall complexity.

Unlike other quantum machine learning solutions, this function is capable of handling large-scale datasets with millions of examples and features without being limited by the number of qubits in the target QPU. The number of qubits only determines the size of the ensemble that can be trained. It is also highly flexible, and can be used to solve classification problems across a wide range of domains, including finance, healthcare, and cybersecurity.

It consistently achieves high accuracies on classically challenging problems involving high-dimensional, noisy, and imbalanced datasets.

![How it works](/images/guides/multiverse-computing-singularity/how_it_works.avif)

It is built for:
1. Engineers and data scientists at companies seeking to enhance their tech offerings by integrating quantum machine learning into their products and services,
2. Researchers at quantum research labs exploring quantum machine learning applications and looking to leverage quantum computing for classification tasks, and
3. Students and teachers at educational institutions in courses like machine learning, and who are looking to demonstrate the advantages of quantum computing.

The following example showcases its various functionalities, including `create`, `list`, `fit`, and `predict`, and demonstrates its usage in a synthetic problem comprising two interleaving half circles, a notoriously challenging problem due to its nonlinear decision boundary.

{/* cspell:ignore quantumly */}

## Function description

This Qiskit Function allows users to solve binary classification problems using Singularity's quantum-enhanced ensemble classifier. Behind the scenes, it uses a hybrid approach to classically train an ensemble of classifiers on the labeled dataset, and then optimize it for maximum diversity and generalization using the Quantum Approximate Optimization Algorithm (QAOA) on IBM&reg; QPUs. Through a user-friendly interface, users can configure a classifier according to their requirements, train it on the dataset of their choice, and use it to make predictions on a previously unseen dataset.

To solve a generic classification problem:
1. Preprocess the dataset, and split it into training and testing sets. Optionally, you can further split the training set into training and validation sets. This can be achieved using [scikit-learn](https://scikit-learn.org/1.5/modules/generated/sklearn.model_selection.train_test_split.html).
2. If the training set is imbalanced, you can resample it to balance the classes using [imbalanced-learn](https://imbalanced-learn.org/stable/introduction.html#problem-statement-regarding-imbalanced-data-sets).
3. Upload the training, validation, and test sets separately to the function's storage using the catalog's `file_upload` method, passing it the relevant path each time.
4. Initialize the quantum classifier by using the function's `create` action, which accepts hyperparameters such as the number and types of learners, the regularization (lambda value), and optimization options including the number of layers, the type of classical optimizer, the quantum backend, and so on.
5. Train the quantum classifier on the training set using the function's `fit` action, passing it the labeled training set, and the validation set if applicable.
6. Make predictions on the previously unseen test set using the function's `predict` action.

## Action-based approach

The function uses an action-based approach. You can think of it as a virtual environment where you use actions to perform tasks or change its state. Currently, it offers the following actions: [list](#1-list), [create](#2-create), [delete](#3-delete), [fit](#4-fit), [predict](#5-predict), [fit_predict](#6-fit-predict), and [create_fit_predict](#7-create-fit-predict). The following example demonstrates the `create_fit_predict` action.

In [1]:
# Import QiskitFunctionsCatalog to load the
# "Singularity Machine Learning - Classification" function by Multiverse Computing
from qiskit_ibm_catalog import QiskitFunctionsCatalog

# Import the make_moons and the train_test_split functions from scikit-learn
# to create a synthetic dataset and split it into training and test datasets
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# authentication
# If you have not previously saved your credentials, follow instructions at
# https://docs.quantum.ibm.com/guides/functions
# to authenticate with your API key.
catalog = QiskitFunctionsCatalog()

# load "Singularity Machine Learning - Classification" function by Multiverse Computing
singularity = catalog.load("multiverse/singularity")

# generate the synthetic dataset
X, y = make_moons(n_samples=1000)

# split the data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

job = singularity.run(
    action="create_fit_predict",
    num_learners=10,
    regularization=0.01,
    optimizer_options={"simulator": True},
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    options={"save": False},
)

# get job status and result
status = job.status()
result = job.result()

print("Job status: ", status)
print("Action result status: ", result["status"])
print("Action result message: ", result["message"])
print("Predictions (first five results): ", result["data"]["predictions"][:5])
print(
    "Probabilities (first five results): ",
    result["data"]["probabilities"][:5],
)

Job status:  QUEUED
Action result status:  ok
Action result message:  Classifier created, fitted, and predicted.
Predictions (first five results):  [0, 1, 0, 0, 0]
Probabilities (first five results):  [[0.7975136940667589, 0.20248630593324138], [0.08459026150938889, 0.9154097384906114], [0.8043402543461567, 0.19565974565384334], [0.7975136940667589, 0.20248630593324138], [0.7975136940667589, 0.20248630593324138]]


### 1. List

The `list` action retrieves all stored classifiers in `*.pkl.tar` format from the shared data directory. You can also access the contents of this directory by using the `catalog.files()` method. In general, the list action searches for files with the `*.pkl.tar` extension in the shared data directory and returns them in a list format.

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |

#### Usage

In [ ]:
job = singularity.run(action="list")

### 2. Create

The `create` action creates a classifier of the specified `quantum_classifier` type by using the provided parameters, and saves it in the shared data directory.

<Admonition type="note">
The function currently supports only the `QuantumEnhancedEnsembleClassifier`.
</Admonition>

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the quantum classifier, e.g., "spam_classifier". | Yes |
| quantum_classifier | string | The type of the quantum classifier, i.e., "QuantumEnhancedEnsembleClassifier". Default is "QuantumEnhancedEnsembleClassifier". | No |
| num_learners | integer | The number of learners in the ensemble. Default is set to a small value of 10 to facilitate quick training and optimization. | No |
| learners_types | list | Types of learners. Among supported types are: `DecisionTreeClassifier`, `GaussianNB`, `KNeighborsClassifier`, `MLPClassifier`, and `LogisticRegression`. Further details related to each can be found in the [scikit-learn documentation](https://scikit-learn.org/stable/supervised_learning.html). Default is [`DecisionTreeClassifier`]. | No |
| learners_proportions | list | Proportions of each learner type in the ensemble. Default is: `[1.0]`. | No |
| learners_options | list | Options for each learner type in the ensemble. For a complete list of options corresponding to the chosen learner type/s, consult [scikit-learn documentation](https://scikit-learn.org/stable/supervised_learning.html). Default is `[{"max_depth": 3, "splitter": "random", "class_weight": None}]`. | No |
| regularization | float | Regularization parameter. Default is 0.01. | No |
| weight_update_method | string | Method for update of sample weights from among "logarithmic" and "quadratic". Default is "logarithmic". | No |
| sample_scaling | boolean | Whether sample scaling should be applied. Default is False. | No |
| prediction_scaling | float | Scaling factor for predictions. Default is None. | No |
| optimizer_options | dictionary | QAOA optimizer options. A list of available options is presented later in this documentation. | No |
| voting | string | Use majority voting ("hard") or average of probabilities ("soft") for aggregating learners' predictions/probabilities. Default is "hard". | No |
| prob_threshold | float | Optimal probability threshold. Default is 0.5. | No |
| instance | string | IBM instance. Default is None. | No |
| backend_name | string | IBM compute resource. Default is None, which means the backend with the fewest pending jobs will be used. | No |
| random_state | integer | Control randomness for repeatability. Default is None. | No |

{/* cspell:ignore sparsify, sparsification */}

- Additionally, `optimizer_options` are enlisted as follows:

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| num_solutions | integer | The number of solutions. Default is 1024. | No |
| reps | integer | The number of repetitions. Default is 4. | No |
| sparsify | float | The sparsification threshold. Default is 0.001. | No |
| theta | float | The initial value of theta, a variational parameter of QAOA. Default is None. | No |
| simulator | boolean | Whether to use a simulator or a QPU. Default is False. | No |
| classical_optimizer | string | Name of the classical optimizer for the QAOA. Default is "COBYLA". All solvers offered by SciPy, as enlisted [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize), are usable. You will need to set `classical_optimizer_options` accordingly. | No |
| classical_optimizer_options | dictionary | Classical optimizer options. For a complete list of available options, consult [SciPy documentation](https://docs.scipy.org/doc/scipy/reference/). Default is `{"maxiter": 60}`. | No |
| optimization_level | integer | The depth of the QAOA circuit. Default is 3. | No |
| num_transpiler_runs | integer | Number of transpiler runs. Default is 30. | No |
| pass_manager_options | dictionary | Options for generating preset pass manager. Default is `{"approximation_degree": 1.0}`. | No |
| estimator_options | dictionary | Estimator options. For a complete list of available options, consult [Qiskit Runtime Client documentation](/api/qiskit-ibm-runtime/options-estimator-options). | No |
| sampler_options | dictionary | Sampler options. For a complete list of available options, consult the [Qiskit Runtime Client documentation](/api/qiskit-ibm-runtime/options-sampler-options). | No |

- Default `estimator_options` are:

|     Name    |    Type     | Value  |
|-------------|-------------|-------------|
| default_shots | integer | 1024 |
| resilience_level | integer | 2 |
| twirling | dictionary | `{"enable_gates": True}` |
| dynamical_decoupling | dictionary | `{"enable": True}` |
| resilience_options | dictionary | `{"zne_mitigation": False, "zne": {"amplifier": "pea", "noise_factors": [1.0, 1.3, 1.6], "extrapolator": ["linear", "polynomial_degree_2", "exponential"],}}` |

- Default `sampler_options` are:

|     Name    |    Type     | Value |
|-------------|-------------|-------------|
| twirling | dictionary | `{"enable_gates": True}` |
| dynamical_decoupling | dictionary | `{"enable": True}` |

#### Usage

In [ ]:
job = singularity.run(
    action="create",
    name="classifier_name",  # specify your custom name for the classifier here
    num_learners=10,
    regularization=0.01,
    optimizer_options={"simulator": True},
)

#### Validations

- `name`:
    - The name must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - No classifier with the same name should already exist in the shared data directory.

### 3. Delete

The `delete` action removes a classifier from the shared data directory.

#### Inputs
|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the classifier to delete. | Yes |

#### Usage

In [ ]:
job = singularity.run(
    action="delete",
    name="classifier_name",  # specify the name of the classifier to delete here
)

#### Validations

- `name`:
    - The name must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - A classifier with the same name should already exist in the shared data directory.

### 4. Fit

The `fit` action trains a classifier using the provided training data.

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the classifier to train. | Yes |
| X | array or list or string | The training data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| y | array or list or string | The training target values. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| fit_params | dictionary | Additional parameters to pass to the `fit` method of the classifier. | No |

##### fit_params
|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| validation_data | tuple | The validation data and labels. | No |
| pos_label | integer or string | The class label to be mapped to 1. | No |

#### Usage

In [ ]:
job = singularity.run(
    action="fit",
    name="classifier_name",  # specify the name of the classifier to train here
    X=X_train,  # or "X_train.npy" if you uploaded it in the shared data directory
    y=y_train,  # or "y_train.npy" if you uploaded it in the shared data directory
    fit_params={},  # define the fit parameters here
)

#### Validations

- `name`:
    - The name must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - A classifier with the same name should already exist in the shared data directory.

### 5. Predict

The `predict` action is used to obtain hard and soft predictions (probabilities).

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the classifier to be used. | Yes |
| X | array or list or string | The test data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| options["out"] | string | The output JSON filename to save the predictions in the shared data directory. If not provided, the predictions are returned in the job result. | No |

#### Usage

In [ ]:
job = singularity.run(
    action="predict",
    name="classifier_name",  # specify the name of the classifier to use here
    X=X_test,  # or "X_test.npy" if you uploaded it to the shared data directory
    options={
        "out": "output.json",
    },
)

#### Validations

- `name`:
    - The name must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - A classifier with the same name should already exist in the shared data directory.
- `options["out"]`:
    - The filename must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - It must have the `.json` extension.

### 6. Fit-predict

The `fit_predict` action trains a classifier using the training data and then uses it to obtain hard and soft predictions (probabilities).

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the classifier to be used. | Yes |
| X_train | array or list or string | The training data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| y_train | array or list or string | The training target values. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| X_test | array or list or string | The test data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| fit_params | dictionary | Additional parameters to pass to the `fit` method of the classifier. | No |
| options["out"] | string | The output JSON filename to save the predictions in the shared data directory. If not provided, the predictions are returned in the job result. | No |

#### Usage

In [ ]:
job = singularity.run(
    action="fit_predict",
    name="classifier_name",  # specify the name of the classifier to use here
    X_train=X_train,  # or "X_train.npy" if you uploaded it in the shared data directory
    y_train=y_train,  # or "y_train.npy" if you uploaded it in the shared data directory
    X_test=X_test,  # or "X_test.npy" if you uploaded it in the shared data directory
    fit_params={},  # define the fit parameters here
    options={
        "out": "output.json",
    },
)

#### Validations

- `name`:
    - The name must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - A classifier with the same name should already exist in the shared data directory.

- `options["out"]`:
    - The filename must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - It must have the `.json` extension.

### 7. Create-fit-predict

The `create_fit_predict` action creates a classifier, trains it using the provided training data, and then uses it to obtain hard and soft predictions (probabilities).

#### Inputs

|     Name    |    Type     | Description |   Required  |
|-------------|-------------|-------------|-------------|
| action | string | The name of the action from among "create", "list", "fit", "predict", "fit_predict", "create_fit_predict" and "delete". | Yes |
| name | string | The name of the classifier to be used. | Yes |
| quantum_classifier | string | The type of the classifier, i.e., "QuantumEnhancedEnsembleClassifier". Default is "QuantumEnhancedEnsembleClassifier". | No |
| X_train | array or list or string | The training data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| y_train | array or list or string | The training target values. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| X_test | array or list or string | The test data. This can be a NumPy array, a list, or a string referencing a filename in the shared data directory. | Yes |
| fit_params | dictionary | Additional parameters to pass to the `fit` method of the classifier. | No |
| options["save"] | boolean | Whether to save to trained classifier in the shared data directory. Default is `True`. | No |
| options["out"] | string | The output JSON filename to save the predictions in the shared data directory. If not provided, the predictions are returned in the job result. | No |

#### Usage

In [ ]:
job = singularity.run(
    action="create_fit_predict",
    name="classifier_name",  # specify your custom name for the classifier here
    num_learners=10,
    regularization=0.01,
    optimizer_options={"simulator": True},
    X_train=X_train,  # or "X_train.npy" if you uploaded it in the shared data directory
    y_train=y_train,  # or "y_train.npy" if you uploaded it in the shared data directory
    X_test=X_test,  # or "X_test.npy" if you uploaded it in the shared data directory
    fit_params={},  # define the fit parameters here
    options={
        "save": True,
        "out": "output.json",
    },
)

#### Validations

- `name`:
    - If `options["save"]` is set to `True`:
        - The name must be unique, a string up to 64 characters long.
        - It can only include alphanumeric characters and underscores.
        - It must start with a letter and cannot end with an underscore.
        - No classifier with the same name should already exist in the shared data directory.

- `options["out"]`:
    - The filename must be unique, a string up to 64 characters long.
    - It can only include alphanumeric characters and underscores.
    - It must start with a letter and cannot end with an underscore.
    - It must have the `.json` extension.

---

## Get started

<LegacyContent>
Authenticate using your [IBM Quantum Platform API token](http://quantum.ibm.com/), and select the Qiskit Function as follows:
</LegacyContent>
<CloudContent>
Authenticate using your [IBM Quantum Platform API key](http://quantum.cloud.ibm.com/), and select the Qiskit Function as follows:
</CloudContent>

In [2]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()

# load function
singularity = catalog.load("multiverse/singularity")

## Example

In this example, you'll use the "Singularity Machine Learning - Classification" function to classify a dataset consisting of two interleaving, moon-shaped half-circles. The dataset is synthetic, two-dimensional, and labeled with binary labels. It is created to be challenging for algorithms such as centroid-based clustering and linear classification.

![Moons dataset](/images/guides/multiverse-computing-singularity/moon_shaped.avif)

Through this process, you'll learn how to create the classifier, fit it to the training data, use it to predict on the test data, and delete the classifier when you're finished.

Before starting, you need to install [scikit-learn](https://scikit-learn.org/). Install it using the following command:

```sh
python3 -m pip install scikit-learn
```

Perform the following steps:

1) Create the synthetic dataset using the [make_moons](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) function from [scikit-learn](https://scikit-learn.org/).
2) Upload the generated synthetic dataset to the [shared data directory](https://qiskit.github.io/qiskit-serverless/getting_started/experimental/manage_data_directory.html).
3) Create the quantum-enhanced classifier using the [create](#2-create) action.
4) Enlist your classifiers using the [list](#1-list) action.
5) Train the classifier on the train data using the [fit](#4-fit) action.
6) Use the trained classifier to predict on the test data using the [predict](#5-predict) action.
7) Delete the classifier using the [delete](#3-delete) action.
8) Clean up after you're done.

**Step 1.** Import the necessary modules and generate the synthetic dataset, then split it into training and test datasets.

In [7]:
# import the necessary modules for this example
import os
import tarfile
import numpy as np

# Import the make_moons and the train_test_split functions from scikit-learn
# to create a synthetic dataset and split it into training and test datasets
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

# generate the synthetic dataset
X, y = make_moons(n_samples=10000)

# split the data into training and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# print the first 10 samples of the training dataset
print("Features:", X_train[:10, :])
print("Targets:", y_train[:10])

Features: [[ 6.27427223e-01 -4.28002977e-01]
 [ 1.50566059e-01  9.88599950e-01]
 [ 6.51703130e-01  7.58474146e-01]
 [ 6.64219130e-04  4.63558306e-01]
 [ 1.80634911e+00 -9.14398615e-02]
 [ 5.57234689e-01  8.30355045e-01]
 [ 1.98286424e+00  3.15669108e-01]
 [ 7.38804205e-01 -4.65285842e-01]
 [ 1.98768342e+00  3.43534500e-01]
 [ 7.59101739e-02  1.17824656e-01]]
Targets: [1 0 0 1 1 0 1 1 1 1]


**Step 2.** Save the labeled training and test datasets on your local disk, and then upload them to the [shared data directory](https://qiskit.github.io/qiskit-serverless/getting_started/experimental/manage_data_directory.html).

In [8]:
def make_tarfile(file_path, tar_file_name):
    with tarfile.open(tar_file_name, "w") as tar:
        tar.add(file_path, arcname=os.path.basename(file_path))


# save the training and test datasets on your local disk
np.save("X_train.npy", X_train)
np.save("y_train.npy", y_train)
np.save("X_test.npy", X_test)
np.save("y_test.npy", y_test)

# create tar files for the datasets
make_tarfile("X_train.npy", "X_train.npy.tar")
make_tarfile("y_train.npy", "y_train.npy.tar")
make_tarfile("X_test.npy", "X_test.npy.tar")
make_tarfile("y_test.npy", "y_test.npy.tar")

# upload the datasets to the shared data directory
catalog.file_upload("X_train.npy.tar", singularity)
catalog.file_upload("y_train.npy.tar", singularity)
catalog.file_upload("X_test.npy.tar", singularity)
catalog.file_upload("y_test.npy.tar", singularity)

# view/enlist the uploaded files in the shared data directory
print(catalog.files(singularity))

['X_test.npy.tar', 'X_train.npy.tar', 'y_test.npy.tar', 'y_train.npy.tar']


**Step 3.** Create a quantum-enhanced classifier using the [create](#2-create) action.

In [9]:
job = singularity.run(
    action="create",
    name="my_classifier",
    num_learners=10,
    learners_types=[
        "DecisionTreeClassifier",
        "KNeighborsClassifier",
    ],
    learners_proportions=[0.5, 0.5],
    learners_options=[{}, {}],
    regularization=0.01,
    weight_update_method="logarithmic",
    sample_scaling=True,
    optimizer_options={"simulator": True},
    voting="soft",
    prob_threshold=0.5,
)

print(job.result())

{'status': 'ok', 'message': 'Classifier created.', 'data': {}}


In [10]:
# list available classifiers using the list action
job = singularity.run(action="list")

print(job.result())

# you can also find your classifiers in the shared data directory with a *.pkl.tar extension
print(catalog.files(singularity))

{'status': 'ok', 'message': 'Classifiers listed.', 'data': {'classifiers': ['my_classifier']}}
['X_test.npy.tar', 'X_train.npy.tar', 'my_classifier.pkl.tar', 'y_test.npy.tar', 'y_train.npy.tar']


**Step 4.** Train the quantum-enhanced classifier using the [fit](#4-fit) action.

In [12]:
job = singularity.run(
    action="fit",
    name="my_classifier",
    X="X_train.npy",  # you do not need to specify the tar extension
    y="y_train.npy",  # you do not need to specify the tar extension
)

print(job.result())

{'status': 'ok', 'message': 'Classifier fitted.', 'data': {}}


**Step 5.** Obtain predictions and probabilities from the quantum-enhanced classifier using the [predict](#5-predict) action.

In [13]:
job = singularity.run(
    action="predict",
    name="my_classifier",
    X="X_test.npy",  # you do not need to specify the tar extension
)

result = job.result()

print("Action result status: ", result["status"])
print("Action result message: ", result["message"])
print("Predictions (first five results):", result["data"]["predictions"][:5])
print(
    "Probabilities (first five results):", result["data"]["probabilities"][:5]
)

Action result status:  ok
Action result message:  Classifier predicted.
Predictions (first five results): [1, 0, 1, 1, 1]
Probabilities (first five results): [[0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0]]


**Step 6.** Delete the quantum-enhanced classifier using the [delete](#3-delete) action.

In [14]:
job = singularity.run(
    action="delete",
    name="my_classifier",
)

# or you can delete from the shared data directory
# catalog.file_delete("my_classifier.pkl.tar", singularity)

print(job.result())

{'status': 'ok', 'message': 'Classifier deleted.', 'data': {}}


**Step 7.** Clean up local and shared data directories.

In [ ]:
# delete the numpy files from your local disk
os.remove("X_train.npy")
os.remove("y_train.npy")
os.remove("X_test.npy")
os.remove("y_test.npy")

# delete the tar files from your local disk
os.remove("X_train.npy.tar")
os.remove("y_train.npy.tar")
os.remove("X_test.npy.tar")
os.remove("y_test.npy.tar")

# delete the tar files from the shared data
catalog.file_delete("X_train.npy.tar", singularity)
catalog.file_delete("y_train.npy.tar", singularity)
catalog.file_delete("X_test.npy.tar", singularity)
catalog.file_delete("y_test.npy.tar", singularity)

## Benchmarks

These benchmarks show that the classifier can achieve extremely high accuracies on challenging problems. They also show that increasing the number of learners in the ensemble (number of qubits) can lead to increased accuracy.

"Classical accuracy" refers to the accuracy obtained using corresponding classical state of the art which, in this case, is an AdaBoost classifier based on an ensemble of size 75. "Quantum accuracy", on the other hand, refers to the accuracy obtained using the "Singularity Machine Learning - Classification".

| Problem | Dataset Size | Ensemble Size | Number of Qubits | Classical Accuracy | Quantum Accuracy | Improvement |
|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| Grid stability | 5000 examples, 12 features | 55 | 55 |  76% | 91% | 15% |
| Grid stability | 5000 examples, 12 features | 65 | 65 |  76% | 92% | 16% |
| Grid stability | 5000 examples, 12 features | 75 | 75 |  76% | 94% | 18% |
| Grid stability | 5000 examples, 12 features | 85 | 85 |  76% | 94% | 18% |
| Grid stability | 5000 examples, 12 features | 100 | 100 |  76% | 95% | 19% |

----

As quantum hardware evolves and scales, the implications for our quantum classifier become increasingly significant. While the number of qubits does impose limitations on the size of the ensemble that can be utilized, it does not restrict the volume of data that can be processed. This powerful capability enables the classifier to efficiently handle datasets containing millions of data points and thousands of features. Importantly, the constraints related to ensemble size can be addressed through the implementation of a large-scale version of the classifier. By leveraging an iterative outer-loop approach, the ensemble can be dynamically expanded, enhancing flexibility and overall performance. However, it's worth noting that this feature has not yet been implemented in the current version of the classifier.

## Get support

For any questions, [reach out to Multiverse Computing](mailto:singularity@multiversecomputing.com).

Be sure to include the following information:

- The Qiskit Function Job ID (`job.job_id`)
- A detailed description of the issue
- Any relevant error messages or codes
- Steps to reproduce the issue

## Next steps

<Admonition type="tip" title="Recommendations">

- Request access to [Multiverse Computing's Singularity Machine Learning Classification function](https://quantum.ibm.com/functions).
- Try the [Solve classification problems with Multiverse Computing's Singularity Machine Learning](https://learning.quantum.ibm.com/catalog/tutorials) tutorial.

</Admonition>